In [3]:
# 匯入 Shioaji 套件
import shioaji as sj
import pandas as pd
from shioaji import constant
from datetime import datetime, timedelta
from shioaji import TickFOPv1, BidAskFOPv1, Exchange
import os
import numpy as np
from tqdm import tqdm
import time


2024-12-27 17:54:33.916 | WARNING  | importlib._bootstrap:_call_with_frames_removed:228 - Optional: pip install shioaji[speed] for better performance.


In [4]:
# 建立 Shioaji api 物件
api = sj.Shioaji(simulation=True)

acc = api.login(
    api_key="",
    secret_key=""
)#個人帳戶資料

print(api.usage())

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
connections=1 limit_bytes=524288000 remaining_bytes=524288000


In [5]:

def get_date_range(start_date, end_date):
    """
    生成從 start_date 到 end_date 的日期範圍（包含起始和結束日期）。
    """
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    date_list = []
    while start <= end:
        date_list.append(start.strftime("%Y-%m-%d"))
        start += timedelta(days=1)
    return date_list
def save_kbars_to_csv(api, tic_num, start_date, end_date):
    """
    按天下載 KBAR 資料，進行處理並保存為指定格式的 CSV。
    
    :param api: Shioaji API 實例
    :param tic_num: 股票代碼
    :param start_date: 起始日期 (YYYY-MM-DD)
    :param end_date: 結束日期 (YYYY-MM-DD)
    """
    # 創建根目錄資料夾
    root_dir = f"./{tic_num}_kbars"
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)

    # 根據年份創建子資料夾
    date_range = get_date_range(start_date, end_date)
    progress_bar = tqdm(date_range, desc="Downloading KBAR Data", unit="day")

    for date in progress_bar:
        year = date.split("-")[0]
        year_dir = os.path.join(root_dir, year)
        if not os.path.exists(year_dir):
            os.makedirs(year_dir)

        try:
            progress_bar.set_postfix({"Processing": date})
            
            # 下載當日的 KBAR 資料
            kbars = api.kbars(
                contract=api.Contracts.Stocks[tic_num],
                start=date,
                end=date
            )
            
            # 將 KBAR 資料轉為 DataFrame 並進行處理
            df = pd.DataFrame({**kbars})
            df['tic'] = [f'{tic_num}.TW'] * len(df['ts'])
            df.pop('Amount')  # 移除 Amount 欄位
            df.ts = pd.to_datetime(df.ts)  # 將 ts 轉為 datetime 格式
            df = df.rename(columns={
                'Volume': 'volume',
                'Close': 'close',
                'Open': 'open',
                'High': 'high',
                'Low': 'low',
                'ts': 'date'
            })
            
            # 儲存為 CSV 檔案
            csv_file = os.path.join(year_dir, f"{tic_num}_{date}_1min.csv")
            df.to_csv(csv_file, index=False)
        except Exception as e:
            progress_bar.set_postfix({"Error": f"Failed for {date}"})
            print(f"Failed to download KBAR for {date}: {e}")

        time.sleep(1)  # 避免過於頻繁的請求


In [6]:
# 設定股票代碼與日期範圍
# tic_num = "2454"  # 替換成目標股票代碼
start_date = "2023-01-03"
end_date = "2023-01-03"
num = ["2308"]

for i in num:
    save_kbars_to_csv(api, i, start_date, end_date)